In [7]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

import nltk

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns

from wordcloud import WordCloud, STOPWORDS

from textblob import TextBlob

In [ ]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [2]:
# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

C:\Users\pk\anaconda3\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pk\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [5]:
df3 = pd.read_csv(r"C:/Users/pk/Projects/balakot/public.csv")

In [8]:
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english'))

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\r', ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
df3['tweet'] = df3['text'].apply(clean)

In [26]:
text = df3['tweet']

In [15]:
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

In [16]:
scores

array([0.22724693, 0.751812  , 0.02094114], dtype=float32)

In [14]:
# # TF
# model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
# model.save_pretrained(MODEL)

# text = "Good night 😊"
# encoded_input = tokenizer(text, return_tensors='tf')
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) neutral 0.7518
2) negative 0.2272
3) positive 0.0209


In [29]:
def sentiment_classifier(text):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)


    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    for i in range(scores.shape[0]):
        l = labels[ranking[0]] 
        #print(l)
        s = scores[ranking[0]] 
        #print(f"{i+1}) {l} {np.round(float(s), 4)}")
        classification = l

    
    return classification

In [30]:
sentiment_classifier(text[4])

'neutral'

In [ ]:
df3['sentiment'] = df3['tweet'].apply(sentiment_classifier)

In [32]:
df3

,Unnamed: 0,text,user_username,source,created_at,user_location,user_name,retweet_count,like_count,quote_count,user_tweet_count,user_list_count,user_followers_count,user_following_count,tweet
0,4758,Indian aircrafts intruded from Muzafarabad sec...,OfficialDGISPR,Twitter for iPad,2019-02-26T01:36:30.000Z,Pakistan,DG ISPR,10406,40211,2685,1222,1322,5891405,0,indian aircraft intrud muzafarabad sector face...
1,6693,1000 kgs bombs dropped on terror camps with pi...,majorgauravarya,Twitter for iPhone,2019-02-26T03:49:02.000Z,India,Major Gaurav Arya (Retd),10404,34533,669,9833,647,1415589,367,kgs bomb drop terror camp pin point accuraci ...
2,53217,I visited mountain area of Jabba 25 km away fr...,HamidMirPAK,Twitter for iPhone,2019-02-27T09:38:38.000Z,"Islamabad, Pakistan",Hamid Mir,8184,28732,1178,41320,4030,7557234,488,visit mountain area jabba km away balakot tod...
3,1645,We must recognise the sacrifices of our parami...,RahulGandhi,Twitter for iPhone,2019-02-25T08:15:27.000Z,"12, Tughlak Lane, New Delhi",Rahul Gandhi,7520,25055,450,6331,4772,21467331,275,must recognis sacrific paramilitari forc like ...
4,20358,What an explosive morning ! As india celebrate...,TandonRaveena,Twitter for iPhone,2019-02-26T06:30:04.000Z,Planet Earth,Raveena Tandon,3465,21943,153,22900,1418,2421885,405,explos morn india celebr salut braveheart m...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57910,57907,The IAF_MCC confirms the \r\nthe crash earlier...,QadirQd,IFTTT,2019-02-27T18:54:13.000Z,"Vododara, India",Qadir Qd 🇮🇳,0,0,0,154886,22,4538,92,iafmcc confirm crash earlier morn condol fam...
57911,57908,We 130 crore indian are celebrating we have av...,Global_Bakait,Twitter for Android,2019-02-27T18:54:12.000Z,India,Global-बकैत 👤,0,0,0,22837,4,492,2139,crore indian celebr avang pulwama crore pak ...
57912,57909,Our war is against terrorism &amp; people supp...,Crazy_Ayush1,Twitter for Android,2019-02-27T18:54:06.000Z,"Blockchain, Internet",Ayush Sharma,0,0,0,378,1,282,66,war terror amp peopl support dear indian ...
57913,57911,Response #surgicalstrike #paf #pakistan Salute...,hamzaawanwafa,Twitter for Android,2019-02-27T18:53:10.000Z,NaN,Hamza Awan,0,0,0,92,0,17,216,respons surgicalstrik paf pakistan salut pakis...
